In [ ]:
import pandas as pd
import numpy as np
import sqlite3 # in case of sqlite3 version issues: https://stackoverflow.com/questions/27663224/error-while-opening-places-sqlite-database-firefox
import datetime

In [ ]:
# VARIABLES TO CHANGE ---------------------
places_path = 'path-to-places.sqlite'
offset = 30 # number of days to look back. FF only keeps up to 365 days
url_filter = 'youtube.com/watch'

In [ ]:
con = sqlite3.connect(places_path)
cur = con.cursor()
# places.sqlite: https://kb.mozillazine.org/Places.sqlite
# Schema and relationships help: https://wiki.mozilla.org/images/archive/0/08/20081029023953%21Places.sqlite.schema.pdf

In [ ]:
data = cur.execute("SELECT * FROM sqlite_master WHERE type='table'") # get all the table names

In [ ]:
pd.DataFrame.from_records(data, index=None, exclude=None, columns=None, coerce_float=False, nrows=None)

In [ ]:
data = cur.execute('SELECT * FROM moz_places')
hist = pd.DataFrame.from_records(data)

cols = cur.execute("SELECT * FROM pragma_table_info('moz_places') AS tblInfo")
cols = pd.DataFrame.from_records(cols)

hist.columns = cols[1].values

In [ ]:
hist_visits = cur.execute('SELECT * FROM moz_historyvisits')
hist_visits = pd.DataFrame.from_records(hist_visits)

hist_cols = cur.execute("SELECT * FROM pragma_table_info('moz_historyvisits') AS tblInfo")
hist_cols = pd.DataFrame.from_records(hist_cols)

hist_visits.columns = hist_cols[1].values

In [ ]:
hist.head()

In [ ]:
hist_visits.head()

# Data cleaning

Convert the timestamp and filter for relevant times and data.

In [ ]:
hist_visits_clean = hist_visits.copy()
hist_visits_clean['visit_date'] = hist_visits_clean['visit_date']\
                                        .apply(lambda x: datetime.datetime.fromtimestamp(x / 1e6))

In [ ]:
merged = hist_visits_clean.merge(hist, left_on='place_id', right_on='id') # N-1 relationship, how shouldn't be a problem
merged = merged[merged['url'].str.contains(url_filter)]
merged = merged[merged['visit_date'] >= datetime.datetime.today() - datetime.timedelta(days=offset)]

In [ ]:
merged.tail()

# Aggregation

What are the most visited sites?

In [ ]:
final = pd.DataFrame(merged.groupby(['title'])['id_x'].count().sort_values(ascending=False)).reset_index()

In [ ]:
final